In [1]:
# importing packages

from bokeh.io import output_notebook, output_file, curdoc, push_notebook, show
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, Slider, ColumnDataSource, CustomJS, LabelSet, Label
from bokeh.layouts import row, column, gridplot, widgetbox
from bokeh.models.widgets import Panel, Tabs, Button
from bokeh.models.glyphs import Text
from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.transform import dodge
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead

import matplotlib.pyplot as mplt
import matplotlib.lines as mlines
import time

output_notebook()

import numpy as np
import pandas as pd
from itertools import permutations
from gurobipy import *

import multiprocessing as mp

Loading BokehJS ...

In [2]:
# settings

R = 4 # number of matrices max:8
M = 12 # number of rows max:20
N = 18 # number of columns max:20
step = 0.05

S_i = np.array(range(0,M+1,1))
S_j = np.array(range(0,N+1,1))
S_s = np.array(range(0,4,1)) # for the shape of L

bb = [[]] * R
b = [[]] * R

In [3]:
bb[0] = np.array([[0,0,0,0,0.8,0,0,0,0],
[0,0,2.5,2.5,3.8,2.8,2.8,0,0],
[0,1.3,2.3,0,4.1,0,2.1,1.2,0],
[0,0,2.2,0,0,0,2.0,0,0],
[0,1.1,2.1,2.0,2.0,2.0,2.1,0.8,0],
[0,0,0,0,1.0,0,0,0,0]])

In [4]:
bb[1] = np.array([[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0],
[1.3,0,1.1,0,1.0,0,1.1,0,1.0],
[1.0,0,1.1,3.0,3.0,3.0,1.1,0,1.0],
[0.8,0,1.1,3.0,3.0,3.0,1.1,0,1.0],
[0.8,0,1.1,0,1.0,0,1.1,0,1.0],
[2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0]])

In [5]:
bb[2] = np.array([[0.2,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.2],
[0.2,0.2,1.8,1.5,0,1.3,1.5,0,0.2],
[0.2,0,2.8,0,0,0,2.5,0.2,0.2],
[0.2,0.2,3.8,0,8.2,0,3.5,0,0.2],
[0.2,0,4.8,5.8,6.8,5.5,4.5,0.2,0.2],
[0.2,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.2]])

In [6]:
bb[3] = np.array([[0,0,0,0,0,0,0,2.0,4.0],
[0,1.2,1.2,1.2,1.2,1.2,1.2,1.2,0],
[0,1.2,0,0,0,0,0,1.2,0],
[0,1.2,0,0,0,0,0,1.2,0],
[0,1.2,1.2,1.2,1.2,1.2,1.2,1.2,0],
[0,0,0,0,0,0,0,0,0]])

In [7]:
for r in range(R):
    b[r] = np.array([[0.0] * 18]*12)
    for i in range(12):
        for j in range(18):
            b[r][i][j] = bb[r][i//2,j//2]/2

In [8]:
# parameters

a = [[]] * R

for r in range(R):
    a[r] = np.zeros((M, N+1))
    
for r in range(R):
    for i in range(M):
        for j in range(1, N+1):
            a[r][i][j] = a[r][i][j-1] + b[r][i][j-1]

In [ ]:
def draw_partitions(M, N, V):
    # drawing Matrix
    col = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0',\
           '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8',\
           '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']

    plt = figure(plot_width=500, plot_height=round(500*6/9))

    for k in range(K):
        if len(V[k]) == 4:
            plt.quad(top=[V[k][0]], bottom=[V[k][2]], left=[V[k][1]], right=[V[k][3]], color=col[k%20], alpha = 0.4)
        else:
            if V[k][-1] != 0:
                plt.quad(top=[V[k][0]], bottom=[V[k][4]], left=[V[k][5]], right=[V[k][3]], color=col[k%20], alpha = 0.4)
            if V[k][-1] != 1:
                plt.quad(top=[V[k][4]], bottom=[V[k][2]], left=[V[k][5]], right=[V[k][3]], color=col[k%20], alpha = 0.4)
            if V[k][-1] != 2:
                plt.quad(top=[V[k][4]], bottom=[V[k][2]], left=[V[k][1]], right=[V[k][5]], color=col[k%20], alpha = 0.4)
            if V[k][-1] != 3:
                plt.quad(top=[V[k][0]], bottom=[V[k][4]], left=[V[k][1]], right=[V[k][5]], color=col[k%20], alpha = 0.4)

    for i in range(0,M+1):
        plt.line([0, N], [i, i], line_color='black', line_width=1, alpha = 1)
    for i in range(0,N+1):
        plt.line([i, i], [0, M], line_color='black', line_width=1, alpha = 1)
    show(plt)
    return

In [ ]:
cost = {}
for i1 in S_i[:-1]:
    for j1 in S_j[:-1]:
        for i2 in S_i[i1+1:]:
            for j2 in S_j[j1+1:]:
                tmp = np.zeros(R)
                for r in range(R):
                    for i in range(i1,i2):
                        tmp[r] += a[r][i,j2] - a[r][i,j1]
                cost[i1,j1,i2,j2] = tmp.max()

for i1 in S_i[:-1]:
    for j1 in S_j[:-1]:
        for i2 in S_i[i1+1:]:
            for j2 in S_j[j1+1:]:
                for i3 in S_i[i1+1:i2]:
                    for j3 in S_j[j1+1:j2]:
                        tmp = np.zeros(R)
                        for r in range(R):
                            for i in range(i1,i3):
                                tmp[r] += a[r][i,j3] - a[r][i,j1]
                            for i in range(i3,i2):
                                tmp[r] += a[r][i,j2] - a[r][i,j1]
                        cost[i1,j1,i2,j2,i3,j3,0] = tmp.max()
for i1 in S_i[:-1]:
    for j1 in S_j[:-1]:
        for i2 in S_i[i1+1:]:
            for j2 in S_j[j1+1:]:
                for i3 in S_i[i1+1:i2]:
                    for j3 in S_j[j1+1:j2]:
                        tmp = np.zeros(R)
                        for r in range(R):
                            for i in range(i1,i3):
                                tmp[r] += a[r][i,j2] - a[r][i,j1]
                            for i in range(i3,i2):
                                tmp[r] += a[r][i,j3] - a[r][i,j1]
                        cost[i1,j1,i2,j2,i3,j3,1] = tmp.max()                        
for i1 in S_i[:-1]:
    for j1 in S_j[:-1]:
        for i2 in S_i[i1+1:]:
            for j2 in S_j[j1+1:]:
                for i3 in S_i[i1+1:i2]:
                    for j3 in S_j[j1+1:j2]:
                        tmp = np.zeros(R)
                        for r in range(R):
                            for i in range(i1,i3):
                                tmp[r] += a[r][i,j2] - a[r][i,j1]
                            for i in range(i3,i2):
                                tmp[r] += a[r][i,j2] - a[r][i,j3]
                        cost[i1,j1,i2,j2,i3,j3,2] = tmp.max()
for i1 in S_i[:-1]:
    for j1 in S_j[:-1]:
        for i2 in S_i[i1+1:]:
            for j2 in S_j[j1+1:]:
                for i3 in S_i[i1+1:i2]:
                    for j3 in S_j[j1+1:j2]:
                        tmp = np.zeros(R)
                        for r in range(R):
                            for i in range(i1,i3):
                                tmp[r] += a[r][i,j2] - a[r][i,j3]
                            for i in range(i3,i2):
                                tmp[r] += a[r][i,j2] - a[r][i,j1]
                        cost[i1,j1,i2,j2,i3,j3,3] = tmp.max()
# print('0 - Time = {}'.format(round(time.time()-Time0, 1)))

In [ ]:
for K in range(1,16): # number of partitions

    Si = np.array(range(0,M,1))
    Sj = np.array(range(0,N+1,1))
    Sk = np.array(range(0,K+1,1))
    Sr = np.array(range(0,R,1))

    TimeTotal = time.time()
    Time0 = time.time()

    lb = 0
    for r in range(R):
        tmp = b[r].sum()
        if tmp > lb:
            lb = tmp
    lb = lb/K

    cont = True
    it = 1
    # print('**** {} ****'.format(lb))

    # ------------------------------- 1
    Time1 = time.time()
    # model
    BCP = Model('L shape model')
    # variables
    x = BCP.addVars(S_i, S_j, S_i, S_j, S_i, S_j, S_s, lb=0.0, ub=1.0, vtype='B', name="X")
    xx = BCP.addVars(S_i, S_j, S_i, S_j, lb=0.0, ub=1.0, vtype='B', name="X")
    OBJ = BCP.addVar(lb=0.0, vtype='C', name="OBJECTIVE")
    BCP.update()
    # print('1 - Time = {}'.format(round(time.time()-Time1, 1)))

    # ------------------------------- 2
    Time = time.time()
    # constraint 1
    for i1 in S_i:
        for j1 in S_j:
            for i2 in S_i[:i1+1]:
                for j2 in S_j:
                    BCP.remove(xx[i1,j1,i2,j2])
                    for i3 in S_i:
                        for j3 in S_j:
                            for s in S_s:
                                BCP.remove(x[i1,j1,i2,j2,i3,j3,s])
    for i1 in S_i:
        for j1 in S_j:
            for i2 in S_i[i1+1:]:
                for j2 in S_j[:j1+1]:
                    BCP.remove(xx[i1,j1,i2,j2])
                    for i3 in S_i:
                        for j3 in S_j:
                            for s in S_s:
                                BCP.remove(x[i1,j1,i2,j2,i3,j3,s])

    for i1 in S_i:
        for j1 in S_j:
            for i2 in S_i[i1+1:]:
                for j2 in S_j[j1+1:]:
                    for i3 in S_i[:i1+1]:
                        for j3 in S_j:
                            for s in S_s:
                                BCP.remove(x[i1,j1,i2,j2,i3,j3,s])
    for i1 in S_i:
        for j1 in S_j:
            for i2 in S_i[i1+1:]:
                for j2 in S_j[j1+1:]:
                    for i3 in S_i[i2:]:
                        for j3 in S_j:
                            for s in S_s:
                                BCP.remove(x[i1,j1,i2,j2,i3,j3,s])
    for i1 in S_i:
        for j1 in S_j:
            for i2 in S_i[i1+1:]:
                for j2 in S_j[j1+1:]:
                    for i3 in S_i[i1+1:i2]:
                        for j3 in S_j[:j1+1]:
                            for s in S_s:
                                BCP.remove(x[i1,j1,i2,j2,i3,j3,s])                            
    for i1 in S_i:
        for j1 in S_j:
            for i2 in S_i[i1+1:]:
                for j2 in S_j[j1+1:]:
                    for i3 in S_i[i1+1:i2]:
                        for j3 in S_j[j2:]:
                            for s in S_s:
                                BCP.remove(x[i1,j1,i2,j2,i3,j3,s])                              

    BCP.update()
    # print('2 - Time = {}'.format(round(time.time()-Time, 1)))

    # ------------------------------- 4
    Time = time.time()
    # constraint 3
    BCP.addConstr((
        quicksum(x[i1,j1,i2,j2,i3,j3,s]
                 for i1 in S_i[:-1]
                 for j1 in S_j[:-1]
                 for i2 in S_i[i1+1:]
                 for j2 in S_j[j1+1:]
                 for i3 in S_i[i1+1:i2]
                 for j3 in S_j[j1+1:j2]
                 for s in S_s) +     
        quicksum(xx[i1,j1,i2,j2]
                 for i1 in S_i[:-1]
                 for j1 in S_j[:-1]
                 for i2 in S_i[i1+1:]
                 for j2 in S_j[j1+1:]) == K),
        name = "# of partitions")
    BCP.update()
    # print('4 - Time = {}'.format(round(time.time()-Time, 1)))

    # ------------------------------- 5
    Time = time.time()
    BCP.addConstrs((
        quicksum(xx[i1,j1,i2,j2]
                            for i1 in S_i[:I+1]
                            for j1 in S_j[:J+1]
                            for i2 in S_i[I+1:]
                            for j2 in S_j[J+1:])
        + quicksum(x[i1,j1,i2,j2,i3,j3,s]
                            for i1 in S_i[:I+1]
                            for j1 in S_j[:J+1]
                            for i2 in S_i[I+1:]
                            for j2 in S_j[J+1:]
                            for i3 in S_i[i1+1: i2]
                            for j3 in S_j[j1+1: j2]
                            for s in S_s)
        - quicksum(x[i1,j1,i2,j2,i3,j3,0]
                            for i1 in S_i[:I+1]
                            for j1 in S_j[:J+1]
                            for i2 in S_i[I+1:]
                            for j2 in S_j[J+1:]
                            for i3 in S_i[I+1: i2]
                            for j3 in S_j[j1+1: J+1])
        - quicksum(x[i1,j1,i2,j2,i3,j3,1]
                            for i1 in S_i[:I+1]
                            for j1 in S_j[:J+1]
                            for i2 in S_i[I+1:]
                            for j2 in S_j[J+1:]
                            for i3 in S_i[i1+1: I+1]
                            for j3 in S_j[j1+1: J+1])
        - quicksum(x[i1,j1,i2,j2,i3,j3,2]
                            for i1 in S_i[:I+1]
                            for j1 in S_j[:J+1]
                            for i2 in S_i[I+1:]
                            for j2 in S_j[J+1:]
                            for i3 in S_i[i1+1: I+1]
                            for j3 in S_j[J+1: j2])
        - quicksum(x[i1,j1,i2,j2,i3,j3,3]
                            for i1 in S_i[:I+1]
                            for j1 in S_j[:J+1]
                            for i2 in S_i[I+1:]
                            for j2 in S_j[J+1:]
                            for i3 in S_i[I+1: i2]
                            for j3 in S_j[J+1: j2])
                            == 1
                  for I in S_i[:-1]
                  for J in S_j[:-1]),
                  name = "single cell cover")
    BCP.update()
    # print('5 - Time = {}'.format(round(time.time()-Time, 1)))

    # ------------------------------- 6
    Time = time.time()
    # constraint Obj
    BCP.addConstrs((
        OBJ >= xx[i1,j1,i2,j2] * cost[i1,j1,i2,j2]
        for i1 in S_i
        for j1 in S_j
        for i2 in S_i[i1+1:]
        for j2 in S_j[j1+1:]),
        name = "OBJ on Sum of the partition")

    BCP.addConstrs((
        OBJ >= x[i1,j1,i2,j2,i3,j3,s] * cost[i1,j1,i2,j2,i3,j3,s]
        for i1 in S_i
        for j1 in S_j
        for i2 in S_i[i1+1:]
        for j2 in S_j[j1+1:]
        for i3 in S_i[i1+1:i2]
        for j3 in S_j[j1+1:j2]
        for s in S_s),
        name = "OBJ on Sum of the partition L")

    BCP.update()
    # print('6 - Time = {}'.format(round(time.time()-Time, 1)))
    # print('\nInitializing - Time = {}\n'.format(round(time.time()-Time1, 1)))

    while cont:
        # ------------------------------- 3
        Time2 = time.time()
        lb_const1 = BCP.addConstrs((
                    xx[i1,j1,i2,j2] == 0
                    for i1 in S_i
                    for j1 in S_j
                    for i2 in S_i[i1+1:]
                    for j2 in S_j[j1+1:]
                    if cost[i1,j1,i2,j2] > lb),
                    name = "UB1")

        lb_const2 = BCP.addConstrs((
                    x[i1,j1,i2,j2,i3,j3,s] == 0
                    for i1 in S_i
                    for j1 in S_j
                    for i2 in S_i[i1+1:]
                    for j2 in S_j[j1+1:]
                    for i3 in S_i[i1+1:i2]
                    for j3 in S_j[j1+1:j2]
                    for s in S_s
                    if cost[i1,j1,i2,j2,i3,j3,s] > lb),
                    name = "UB2")

        BCP.update()
    #     print('3 - Time = {}'.format(round(time.time()-Time2, 1)))

        # ------------------------------- 7
        Time = time.time()
        BCP.setParam('OutputFlag', 0)
        # objective
        BCP.setObjective(OBJ, GRB.MINIMIZE)
        # solution
        BCP.optimize()
    #     print('\n7 - Time = {}'.format(round(time.time()-Time, 1)))

        # ------------------------------- closing    
        if BCP.Status == 2:
            cont = False
    #         print('\nIteration {} - Time = {}\n\n\n'.format(it, round(time.time()-Time2, 1)))
        else:
            lb = lb * (1+step)
            BCP.remove(lb_const1)
            BCP.remove(lb_const2)
    #         print('**** {} ****'.format(lb))    
    #         print('\nIteration {} - Time = {}\n\n\n'.format(it, round(time.time()-Time2, 1)))
            it += 1

    ttime = time.time()-TimeTotal
    # print('\n\n\nTotal - Time = {}     Obj = {}'.format(round(ttime, 1), round(BCP.ObjVal, 2)))



    V = []
    for i1 in S_i[:-1]:
        for j1 in S_j[:-1]:
            for i2 in S_i[i1+1:]:
                for j2 in S_j[j1+1:]:
                    if xx[i1,j1,i2,j2].X > 0.001:
                        V.append((i1,j1,i2,j2))
                    for i3 in S_i[i1+1:i2]:
                        for j3 in S_j[j1+1:j2]:
                            for s in S_s:
                                if x[i1,j1,i2,j2,i3,j3,s].X > 0.001:
                                    V.append((i1,j1,i2,j2,i3,j3,s))

    print(K, BCP.ObjVal, round(ttime,1))
    draw_partitions(M, N, V)

    R_name = ['MECHANICAL', 'FRAMING', 'ELECTRICAL', 'OVERHEAD PLUMBING']

    col_rct = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0',\
           '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8',\
           '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']

    col = ['#000075','#42d4f4','#f58231','#8A2BE2','#A52A2A','#f032e6','#4363d8','#808000', 'green']

    takt = np.zeros(R*K).reshape(K,R)
    for i, v in enumerate(V):
        if len(v) == 4:
            for t in range(R):
                tmp = 0
                for row in range(v[0],v[2]):
                    tmp += a[t][row,v[3]] - a[t][row,v[1]]
                    takt[i][t] = tmp
        elif len(v) == 7:
            for t in range(R):
                tmp = 0
                for row in range(v[0],v[2]):
                        tmp += a[t][row,v[3]] - a[t][row,v[1]]  
                if v[6] == 0:
                    for row in range(v[0],v[4]):
                        tmp -= a[t][row,v[3]] - a[t][row,v[5]]
                elif v[6] == 3:
                    for row in range(v[0],v[4]):
                        tmp -= a[t][row,v[5]] - a[t][row,v[1]]                
                elif v[6] == 2:
                    for row in range(v[4],v[2]):
                        tmp -= a[t][row,v[5]] - a[t][row,v[1]]                
                elif v[6] == 1:
                    for row in range(v[4],v[2]):
                        tmp -= a[t][row,v[3]] - a[t][row,v[5]]                
                else:
                    print('***** Error q *****')
                takt[i][t] = tmp   
        else:
            print('****** Error on len(v) ******')

    partitions = []
    for i in range(1,K+1):
        partitions.append("Partition_" + str(i))

    data = {}
    data['prtitions'] = partitions
    for r in range(R):
        data[R_name[r]] = []
        for k in range(K):
            data[R_name[r]].append(takt[k,r])

    w = 2.8/(5*R-1)
    source = ColumnDataSource(data=data)

    p = figure(x_range=partitions, y_range=(0, BCP.ObjVal*1.5), plot_height=350, plot_width=1250,\
               title="TAKT planning result", tools="save")

    for k in range(K):
        p.quad(top=[BCP.ObjVal*1.3], bottom=[0], left=[k], right=[k+1], color=col_rct[k], alpha = 0.2)    

    for r in range(R):
        p.vbar(x=dodge('prtitions', w*5/4*r-0.35+w/2, range=p.x_range), top=R_name[r], width=w, source=source,
           color=col[r], legend=value(R_name[r]))

    p.x_range.range_padding = 0.1
    p.xgrid.grid_line_color = None
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"

    show(p)
    print('\n--------------------------------------------------------------------------------------\n')

Academic license - for non-commercial use only
